# 📄 PDF Processing Notebook

This notebook demonstrates PDF-specific processing for the Digitize Notes pipeline.

## Features:
1. **PDF to Image Conversion** - Split multi-page PDFs to high-res images
2. **Page Detection** - Identify page boundaries and orientations
3. **Batch Processing** - Process all pages with progress tracking
4. **Metadata Extraction** - Extract PDF metadata (author, title, etc.)

## Prerequisites:
```bash
pip install pdf2image PyMuPDF pillow
# Also need poppler installed:
# brew install poppler  # macOS
# apt-get install poppler-utils  # Linux
```

In [ ]:
import sys
import os

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
sys.path.insert(0, project_root)
sys.path.insert(0, os.path.join(project_root, 'backend/ai-service'))

import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import json
from dataclasses import dataclass, asdict, field
from typing import List, Dict, Any, Tuple, Optional
import uuid
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 10]

print(f"✅ Project root: {project_root}")

---
## 1️⃣ PDF Loading & Metadata Extraction

In [ ]:
try:
    import fitz  # PyMuPDF
    PYMUPDF_AVAILABLE = True
    print("✅ PyMuPDF available")
except ImportError:
    PYMUPDF_AVAILABLE = False
    print("⚠️ PyMuPDF not installed - pip install PyMuPDF")

try:
    from pdf2image import convert_from_path
    PDF2IMAGE_AVAILABLE = True
    print("✅ pdf2image available")
except ImportError:
    PDF2IMAGE_AVAILABLE = False
    print("⚠️ pdf2image not installed - pip install pdf2image")

In [ ]:
@dataclass
class PDFMetadata:
    """Extracted PDF metadata"""
    title: str = ""
    author: str = ""
    subject: str = ""
    creator: str = ""
    producer: str = ""
    creation_date: str = ""
    modification_date: str = ""
    page_count: int = 0
    file_size_bytes: int = 0
    encrypted: bool = False

@dataclass
class PDFPage:
    """A single page from a PDF"""
    page_index: int
    width: int
    height: int
    dpi: int
    rotation: int = 0
    image: np.ndarray = None
    text_content: str = ""
    has_images: bool = False

@dataclass 
class PDFDocument:
    """Complete PDF document with pages"""
    document_id: str
    file_path: str
    metadata: PDFMetadata
    pages: List[PDFPage] = field(default_factory=list)

def extract_pdf_metadata(pdf_path: str) -> PDFMetadata:
    """Extract metadata from PDF using PyMuPDF"""
    if not PYMUPDF_AVAILABLE:
        return PDFMetadata()
    
    doc = fitz.open(pdf_path)
    meta = doc.metadata
    
    return PDFMetadata(
        title=meta.get('title', ''),
        author=meta.get('author', ''),
        subject=meta.get('subject', ''),
        creator=meta.get('creator', ''),
        producer=meta.get('producer', ''),
        creation_date=meta.get('creationDate', ''),
        modification_date=meta.get('modDate', ''),
        page_count=len(doc),
        file_size_bytes=os.path.getsize(pdf_path),
        encrypted=doc.is_encrypted
    )

# Test with a sample PDF
sample_pdf = "../sample.pdf"  # UPDATE THIS PATH

if os.path.exists(sample_pdf):
    metadata = extract_pdf_metadata(sample_pdf)
    print("📄 PDF Metadata:")
    print(f"   Title: {metadata.title}")
    print(f"   Author: {metadata.author}")
    print(f"   Pages: {metadata.page_count}")
    print(f"   Size: {metadata.file_size_bytes / 1024:.1f} KB")
else:
    print(f"⚠️ No sample PDF at {sample_pdf}")
    print("   Create or link a PDF to test")

---
## 2️⃣ PDF to Image Conversion

In [ ]:
class PDFProcessor:
    """Convert PDF pages to high-resolution images"""
    
    def __init__(self, dpi: int = 300, use_pymupdf: bool = True):
        self.dpi = dpi
        self.use_pymupdf = use_pymupdf and PYMUPDF_AVAILABLE
        
    def convert_to_images(
        self, 
        pdf_path: str,
        page_indices: List[int] = None,
        progress_callback: callable = None
    ) -> List[PDFPage]:
        """
        Convert PDF pages to images
        
        Args:
            pdf_path: Path to PDF file
            page_indices: Specific pages to convert (None = all)
            progress_callback: Optional callback(current, total)
            
        Returns:
            List of PDFPage objects with images
        """
        if self.use_pymupdf:
            return self._convert_with_pymupdf(pdf_path, page_indices, progress_callback)
        elif PDF2IMAGE_AVAILABLE:
            return self._convert_with_pdf2image(pdf_path, page_indices, progress_callback)
        else:
            raise RuntimeError("No PDF conversion backend available")
    
    def _convert_with_pymupdf(
        self, 
        pdf_path: str,
        page_indices: List[int],
        progress_callback: callable
    ) -> List[PDFPage]:
        """Convert using PyMuPDF (faster)"""
        doc = fitz.open(pdf_path)
        pages = []
        
        indices = page_indices or range(len(doc))
        total = len(indices) if hasattr(indices, '__len__') else doc.page_count
        
        for i, page_idx in enumerate(indices):
            page = doc[page_idx]
            
            # Calculate matrix for desired DPI
            zoom = self.dpi / 72  # PDF default is 72 DPI
            mat = fitz.Matrix(zoom, zoom)
            
            # Render page to image
            pix = page.get_pixmap(matrix=mat)
            
            # Convert to numpy array
            img = np.frombuffer(pix.samples, dtype=np.uint8)
            img = img.reshape(pix.height, pix.width, pix.n)
            
            # Convert RGB to BGR for OpenCV
            if pix.n == 4:  # RGBA
                img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
            elif pix.n == 3:  # RGB
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            
            # Extract text content (already in PDF)
            text = page.get_text()
            
            # Check for images
            has_images = len(page.get_images()) > 0
            
            pages.append(PDFPage(
                page_index=page_idx,
                width=pix.width,
                height=pix.height,
                dpi=self.dpi,
                rotation=page.rotation,
                image=img,
                text_content=text,
                has_images=has_images
            ))
            
            if progress_callback:
                progress_callback(i + 1, total)
        
        doc.close()
        return pages
    
    def _convert_with_pdf2image(
        self,
        pdf_path: str,
        page_indices: List[int],
        progress_callback: callable
    ) -> List[PDFPage]:
        """Convert using pdf2image (uses Poppler)"""
        from pdf2image import convert_from_path, pdfinfo_from_path
        
        info = pdfinfo_from_path(pdf_path)
        total_pages = info['Pages']
        
        # Convert specific pages or all
        if page_indices:
            # pdf2image uses 1-based indexing
            first_page = min(page_indices) + 1
            last_page = max(page_indices) + 1
        else:
            first_page = 1
            last_page = total_pages
            page_indices = list(range(total_pages))
        
        # Convert
        pil_images = convert_from_path(
            pdf_path,
            dpi=self.dpi,
            first_page=first_page,
            last_page=last_page
        )
        
        pages = []
        for i, (page_idx, pil_img) in enumerate(zip(page_indices, pil_images)):
            # Convert PIL to OpenCV format
            img = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
            
            pages.append(PDFPage(
                page_index=page_idx,
                width=pil_img.width,
                height=pil_img.height,
                dpi=self.dpi,
                rotation=0,
                image=img,
                text_content="",  # pdf2image doesn't extract text
                has_images=True  # Unknown
            ))
            
            if progress_callback:
                progress_callback(i + 1, len(page_indices))
        
        return pages

# Initialize processor
pdf_processor = PDFProcessor(dpi=300)
print(f"✅ PDF Processor initialized")
print(f"   Backend: {'PyMuPDF' if pdf_processor.use_pymupdf else 'pdf2image'}")
print(f"   DPI: {pdf_processor.dpi}")

In [ ]:
# Convert sample PDF
if os.path.exists(sample_pdf):
    def progress(current, total):
        print(f"\r   Processing page {current}/{total}...", end="")
    
    print("📄 Converting PDF to images...")
    start_time = time.time()
    
    pages = pdf_processor.convert_to_images(
        sample_pdf,
        page_indices=None,  # All pages
        progress_callback=progress
    )
    
    elapsed = time.time() - start_time
    print(f"\n✅ Converted {len(pages)} pages in {elapsed:.2f}s")
    
    # Show first page
    if pages:
        first_page = pages[0]
        print(f"\n📃 First Page:")
        print(f"   Size: {first_page.width}x{first_page.height}")
        print(f"   DPI: {first_page.dpi}")
        print(f"   Text length: {len(first_page.text_content)} chars")
        
        plt.figure(figsize=(10, 12))
        plt.imshow(cv2.cvtColor(first_page.image, cv2.COLOR_BGR2RGB))
        plt.title(f"Page 1 ({first_page.width}x{first_page.height} @ {first_page.dpi} DPI)")
        plt.axis('off')
        plt.show()
else:
    print("⚠️ No sample PDF available")

---
## 3️⃣ Batch Processing Pipeline

In [ ]:
from app.services.image_enhancer import ImageEnhancer, A4_SIZE
from app.services.ocr_service import OCRService

class PDFDigitizationPipeline:
    """Complete pipeline for digitizing PDF documents"""
    
    def __init__(
        self,
        dpi: int = 300,
        enhance_images: bool = True,
        extract_text: bool = True
    ):
        self.pdf_processor = PDFProcessor(dpi=dpi)
        self.image_enhancer = ImageEnhancer() if enhance_images else None
        self.ocr_service = OCRService() if extract_text else None
        self.enhance_images = enhance_images
        self.extract_text = extract_text
    
    def process_pdf(
        self,
        pdf_path: str,
        output_dir: str = None,
        progress_callback: callable = None
    ) -> PDFDocument:
        """
        Process entire PDF document
        
        Returns PDFDocument with all pages processed
        """
        doc_id = f"doc_{uuid.uuid4().hex[:8]}"
        
        # Extract metadata
        metadata = extract_pdf_metadata(pdf_path)
        
        # Convert to images
        pages = self.pdf_processor.convert_to_images(pdf_path)
        
        # Process each page
        processed_pages = []
        for i, page in enumerate(pages):
            if progress_callback:
                progress_callback("processing", i + 1, len(pages))
            
            # Enhance image
            if self.enhance_images and self.image_enhancer:
                result = self.image_enhancer.enhance(page.image, crop_document=False)
                page.image = result.enhanced
            
            # Extract text via OCR if not already present
            if self.extract_text and self.ocr_service:
                if not page.text_content.strip():
                    ocr_result = self.ocr_service.extract_text(page.image)
                    page.text_content = ocr_result.text
            
            processed_pages.append(page)
            
            # Save if output dir specified
            if output_dir:
                os.makedirs(output_dir, exist_ok=True)
                cv2.imwrite(
                    os.path.join(output_dir, f"{doc_id}_p{i:03d}.png"),
                    page.image
                )
        
        return PDFDocument(
            document_id=doc_id,
            file_path=pdf_path,
            metadata=metadata,
            pages=processed_pages
        )

# Initialize pipeline
pipeline = PDFDigitizationPipeline(
    dpi=300,
    enhance_images=True,
    extract_text=True
)
print("✅ PDF Digitization Pipeline ready")

In [ ]:
# Process sample PDF
if os.path.exists(sample_pdf):
    def progress(step, current, total):
        print(f"\r   {step}: {current}/{total}", end="")
    
    print("🔄 Processing PDF...")
    start_time = time.time()
    
    document = pipeline.process_pdf(
        sample_pdf,
        output_dir="./output/pdf_test",
        progress_callback=progress
    )
    
    elapsed = time.time() - start_time
    print(f"\n\n✅ Processed in {elapsed:.2f}s")
    print(f"   Document ID: {document.document_id}")
    print(f"   Pages: {len(document.pages)}")
    
    # Show text from each page
    print("\n📄 Extracted Text (first 200 chars per page):")
    for page in document.pages:
        text_preview = page.text_content[:200].replace('\n', ' ')
        print(f"   Page {page.page_index}: {text_preview}...")
else:
    print("⚠️ No sample PDF - create one to test the pipeline")

---
## 4️⃣ Export & Storage

Save processed document data for the API layer.

In [ ]:
def export_document(document: PDFDocument, output_dir: str) -> Dict[str, Any]:
    """Export document to JSON and save images"""
    os.makedirs(output_dir, exist_ok=True)
    
    # Build metadata structure (without images)
    doc_json = {
        "document_id": document.document_id,
        "file_path": document.file_path,
        "metadata": asdict(document.metadata),
        "pages": []
    }
    
    for page in document.pages:
        page_id = f"{document.document_id}_p{page.page_index:03d}"
        
        # Save image
        img_path = os.path.join(output_dir, f"{page_id}.png")
        cv2.imwrite(img_path, page.image)
        
        # Create thumbnail
        thumb_h = 200
        scale = thumb_h / page.height
        thumb_w = int(page.width * scale)
        thumbnail = cv2.resize(page.image, (thumb_w, thumb_h))
        thumb_path = os.path.join(output_dir, f"{page_id}_thumb.png")
        cv2.imwrite(thumb_path, thumbnail)
        
        doc_json["pages"].append({
            "page_id": page_id,
            "page_index": page.page_index,
            "width": page.width,
            "height": page.height,
            "dpi": page.dpi,
            "rotation": page.rotation,
            "image_path": img_path,
            "thumbnail_path": thumb_path,
            "text_content": page.text_content,
            "text_length": len(page.text_content),
            "has_images": page.has_images
        })
    
    # Save JSON
    json_path = os.path.join(output_dir, f"{document.document_id}.json")
    with open(json_path, 'w') as f:
        json.dump(doc_json, f, indent=2)
    
    print(f"✅ Exported to {output_dir}")
    print(f"   Metadata: {json_path}")
    print(f"   Pages: {len(document.pages)} images + thumbnails")
    
    return doc_json

# Export if we have a processed document
if 'document' in dir() and document:
    exported = export_document(document, "./output/exported")

---
## 🎯 Summary

This notebook demonstrated:

| Step | Description | Output |
|------|-------------|--------|
| 1 | PDF Metadata | Title, author, page count |
| 2 | PDF → Images | High-res page images |
| 3 | Batch Processing | Enhanced + OCR'd pages |
| 4 | Export | JSON + images for API |

### Integration Points:
- Use `PDFDigitizationPipeline` in the API upload handler
- Connect `export_document` to S3 storage adapter
- Feed text content to embedding service